In [1]:
# Standard library imports
import os
import sys
import warnings

# Third party imports
import numpy as np
import matplotlib.pyplot as plt
import mat73
import rpy2.robjects as robjects
from scipy.signal import butter, find_peaks, lfilter

# Local imports
from src.features.preprocessing import *
from src.visualization.data_vis import *

# Notebook magic
%matplotlib inline

In [2]:
# Load up a file of interest
recording_of_interest = "fitted_data_mjc57-1212-0110.mat"
file_data = mat73.loadmat('/'.join([os.getcwd(), 'data', 'raw', recording_of_interest]))

# Assign spike data constructs
qspike = file_data['qspike'] # trial spike times aligned to maximize likelihood estimate
pyrspike = file_data['pyrspike'] # all pyramidal spike times and labels
intspike = file_data['intspike'] # all interneuron spike times and labels

# Assign theta LFP vectors
theta = file_data['lfp']
theta_time_axis = file_data['time_eeg']

# Determine indices in theta_time_axis where each trial begins
trial_shifts = (file_data['BP']*file_data['fs2']).astype(int)
unshifted_trial_start_indices = (file_data['tp']*file_data['fs2']).astype(int)
trial_start_indices = unshifted_trial_start_indices + trial_shifts

# Summarize neuron population
print(f'\'{recording_of_interest}\' SUMMARY\n')
print(f'\tNumber of pyramidal neurons: {len(list(set(pyrspike[:, 0])))}')
print(f'\tpyramidal start label: {np.amin(pyrspike[:, 0])}')
print(f'\tpyramidal end label: {np.amax(pyrspike[:, 0])}\n')

print(f'\tNumber of interneurons: {len(list(set(intspike[:, 0])))}')
print(f'\tinterneuron start label: {np.amin(intspike[:, 0])}')
print(f'\tinterneuron end label: {np.amax(intspike[:, 0])}')

# Data preparation
print('')
pyr_com_trials, pyr_sep_trials, pyr_com_neuron_labels, pyr_sep_neuron_labels, pyr_sep_trial_labels = prep_pyr_qspike(qspike, intspike, pyrspike)

'fitted_data_mjc57-1212-0110.mat' SUMMARY

	Number of pyramidal neurons: 64
	pyramidal start label: 1.0
	pyramidal end label: 64.0

	Number of interneurons: 8
	interneuron start label: 65.0
	interneuron end label: 72.0

Pyramidal neurons [20.0, 23.0, 29.0, 58.0, 62.0, 63.0] absent from qspike.


In [3]:
# Filter theta and extract full trial epochs

# Filter parameters
filter_order = 2
lowcut = 6 / (file_data['fs2']*0.5)
highcut = 10 / (file_data['fs2']*0.5)

# Create and apply Butterworth bandpass filter
b, a = butter(filter_order, [lowcut, highcut], btype='band') # The IIR
theta_filtered = lfilter(b, a, theta) # Apply the filter

# Theta epoch extraction
trial_duration = 4 # in seconds
trial_N = int(file_data['fs2']*trial_duration) # Number of data points in each trial based on theta sampling frequency

trial_theta_x = np.zeros((trial_start_indices.size, trial_N), dtype=float) # Columns are data, rows are different trials
trial_theta_y = np.zeros((trial_start_indices.size, trial_N), dtype=float) # Columns are data, rows are different trials

# Extract complete epochs
for trial_i, start_i in enumerate(trial_start_indices):
    trial_theta_x[trial_i, :] = theta_time_axis[start_i:start_i + trial_N]
    trial_theta_y[trial_i, :] = theta_filtered[start_i:start_i + trial_N]


In [4]:
# For each individual trial, classify PR using one of several methods

# Control parameters
test = True
test_num_iterations = 20
classification_method = 'PRQ'

# KDE parameters
bandwidth = 0.02
kde_domain = [0, 4]
cut = 3
kde_domain_N = trial_N

# DFA parameters
DFA_window = 0.8 # seconds

# PRQ parameters
prq_boundary_discont_threshold = 0.95
prq_check_analytic_signal = True

# Boolean classification vector
onehot_vector = np.zeros((len(pyr_sep_trials), 2), dtype=bool)

# Iterate over each individual trial
for i, trial_data in enumerate(pyr_sep_trials):
    
    trial_i = int(pyr_sep_trial_labels[i] - 1) # Trial index is one less than trial label
    active_domain_dur = (trial_data[-1] + cut*bandwidth) - (trial_data[0] - cut*bandwidth)
    active_domain_N = int(file_data['fs2']*active_domain_dur)

    kde_x, kde_y = r_spike_kde([trial_data], bandwidth, kde_domain_N, kde_domain)

    if classification_method == 'DFA':
        detuning = dominant_frequency_analysis(
            kde_y[0],
            trial_theta_x[trial_i, :],
            trial_theta_y[trial_i, :],
            DFA_window,
            file_data['fs2']
       )

    if test == True and classification_method == 'DFA':
        print(f'Max detuning: {detuning}')
        PR_plot(kde_y[0], trial_theta_x[trial_i, :], trial_theta_y[trial_i, :])

    if classification_method == 'PRQ':
        prq = compute_PRQ(
            trial_theta_x[trial_i, :],
            trial_theta_y[trial_i, :],
            trial_data,
            file_data['fs2'],
            discont=prq_boundary_discont_threshold,
            check_analytic=prq_check_analytic_signal
        )

    if test == True and i == test_num_iterations:
        break





[0.13389388 1.30424388 1.76479388 1.80434388 1.94084388 2.07189388
 2.50119388 3.97964388 3.98349388 3.99794388]


NameError: name 'analytic_boundaries_plot' is not defined